In [ ]:
import param
import panel as pn
from adhmodel import AdhModel, AdhMesh, AdhSimulation, BoundaryConditions
from adhui import BoundaryConditionsUI, AdhView, LoadModel
import holoviews as hv
import uit
import glob, os
from uit.panel_util import HpcConnection, PbsScriptStage, JobMonitor
from uit import Client, PbsScript, PbsJob
# from adhui.adh_model_ui import AdhViewBasic
from adhui import AdhModelSubmit

hv.extension('bokeh')

In [ ]:
# load the default dataset for this example
load_model = LoadModel()
load_model.panel()

In [ ]:
adh_mod = load_model._load_data()

In [ ]:
# Instantiate the dashboard
# dashboard = AdhView(adh_mod=adh_mod)
dashboard = AdhViewBasic(adh_mod=adh_mod)
# View the dashboard 
# dashboard.panel() # view the dashboard in a notebook cell
# dashboard.panel().show() # view the dashboard as a separate page

================

In [ ]:
# connect to the hpc
connect = HpcConnection(system='onyx')
connect.panel()

In [ ]:
uit_client = connect.next()

In [ ]:
submit = AdhModelSubmit(uit_client=uit_client)
submit.panel()

In [ ]:
job = submit.submit()

In [ ]:
monitor = JobMonitor(uit_client=uit_client, jobs=[job])
monitor.panel()


In [ ]:
class FileManager(param.Parameterized):
    directory = param.String(
        default=os.getcwd(),
        precedence=0.1
    )
    file_keyword = param.String(
        doc='Keyword for file name. Hidden from ui.',
        default='*', 
        precedence=-1
    )
    file_list = param.ListSelector(
        default=[]
    )
    selected_files = param.ListSelector(
        default=[]
    )

    def __init__(self, **params):
        self.cross_selector = pn.widgets.CrossSelector(name='Fruits', value=[], options=self.file_list, width=900)
        super().__init__(**params) 
        
        self._update_files()
   
    @param.depends('directory', watch=True)
    def _update_files(self):

        self.file_list = glob.glob(os.path.join(self.directory, '*' + self.file_keyword + '*'))
        
        self.cross_selector.options = self.file_list
        
    def panel(self):
        return pn.Column(self.param.directory, self.cross_selector, width=700)
    
    @param.output()
    def output(self):
        """Return a list of the values in the right hand box"""
        self.selected_files = self.cross_selector.value
        return self.selected_files

file_manager = FileManager()

In [ ]:
file_manager.panel()

In [ ]:
class FileManagerHPC(FileManager):
    """File manager for HPC applications using the uit client for communication. 
    
    This extension of FileManager does not currently make use of 
    FileManager.root or FileManager.file_keyword
    
    """
    uit_client = param.ClassSelector(
        Client,
        precedence=-1
    )
    def __init__(self, **params):
        if 'uit_client' in params:
            self.directory = str(params['uit_client'].WORKDIR)
                
        super().__init__(**params)
        self._update_files()
    
    @param.depends('directory', watch=True)
    def _update_files(self):
        # get the ls from the client
        ls_df = uit_client.list_dir(
            path=self.directory, 
            parse=True, 
            as_df=True)
        
        # catch for errors returned as dict
        if type(ls_df) is dict:
            raise RuntimeError(f"""
            Request for directory list returned the error: {ls_df['error']} 
              
            Directory requested: {self.directory}
            
            """)
            
        # convert dataframe to file list
        self.file_list = ls_df['path'].to_list()

        # update cross selector widget
        self.cross_selector.options = self.file_list
    
    def panel(self):
        return pn.Column(self.param.directory, self.cross_selector, width=700)
    
    
file_manager_hpc = FileManagerHPC(uit_client=uit_client)

In [ ]:
file_manager_hpc.panel()

In [ ]:
class FileTransfer(param.Parameterized):
    uit_client = param.ClassSelector(
        Client,
        precedence=-1
    )

    from_location = param.ObjectSelector(
        default='topaz', 
        objects=['topaz', 'onyx', 'local'],
        precedence=0.21
    )
    from_directory = param.String(
        precedence=0.22
    )
    to_location = param.ObjectSelector(
        default='topaz', 
        objects=['topaz', 'onyx', 'local'],
        precedence=0.31
    )
    to_directory = param.String(
        precedence=0.32
    )
    file_manager = param.ClassSelector(
        class_=FileManager,
        precedence=0.4
    )
    transfer_button = param.Action(lambda self: self.param.trigger('transfer_button'), label='Transfer', precedence=1.0)

    
    def __init__(self, uit_client, **params):
        
        super().__init__(**params)
        self.uit_client = uit_client or Client()
        self.file_manager = FileManagerHPC(uit_client=self.uit_client)
        
        # adjust to/from based on uit_client
        self.param.from_location.objects = [self.uit_client.system, 'local']
        self.from_location = self.uit_client.system
        self.param.to_location.objects = [self.uit_client.system, 'local']
        self.to_location = self.uit_client.system
        
    @param.depends('transfer_button', watch=True)
    def transfer(self):
        if self.from_location == 'local':
            for local_file in self.file_manager.cross_selector.value:
                self.uit_client.put_file(local_file, self.to_directory)
        elif self.to_location == 'local':
            for remote_file in self.file_manager.cross_selector.value:
                print('transferring {}'.format(remote_file))
                self.uit_client.get_file(remote_file, local_path=os.path.join(self.to_directory, os.path.basename(remote_file)))
        else:
            print('HPC to HPC transfers are not supported.')
                
    @param.depends('from_directory', watch=True)
    def _update_file_manager(self):
        """ 
        """
        self.file_manager.directory = self.from_directory
        
    def _from_location(self):
        return pn.Column(self.param.from_location, self.param.from_directory)
        
    @param.depends('from_location', watch=True)
    def _to_location(self):
        remote_dir = str(self.uit_client.WORKDIR)
        local_dir = os.getcwd()
        
        if self.from_location == 'local':
            # set from location and dir
            self.from_directory = local_dir
            
            # set to location and dir
            self.to_location = uit_client.system
            self.to_directory = remote_dir
            
            # set file manager to local manager
            self.file_manager = FileManager()
        else:
            # set to location and dir
            self.to_location = 'local'
            self.to_directory = local_dir
            self.from_directory = remote_dir
            
            # set file manager to hpc manager
            self.file_manager = FileManagerHPC(uit_client=self.uit_client)
            
        # set cross selector directory
        self.file_manager._update_files()
        
    @param.depends('from_directory', watch=True) 
    def panel(self):
        from_box = pn.WidgetBox(
            pn.Column(
                self._from_location,
                pn.Column(self.file_manager.cross_selector)
            )
        )

        to_box = pn.WidgetBox(
            pn.Column(self.param.to_location, self.param.to_directory),
            width=900,
            width_policy='max'
        )
        
        return pn.Column(
            from_box,
            to_box,
            pn.panel(self.param.transfer_button)
        )
    
    

In [ ]:
file_transfer = FileTransfer(uit_client=uit_client)
file_transfer.panel()